In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    """
    Envia a imagem, user_id e o nome para o servidor usando multipart/form-data.
    """
    data = {'user_id': user_id, 'name': person_name}

    try:
        # Abrir a imagem detectada e enviá-la com a solicitação POST
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code in [200, 201]:
            print(f"Imagem enviada com sucesso: {image_path}")
            response_json = response.json()
            
            # Exibir a mensagem do servidor (se houver)
            if 'message' in response_json:
                print(f"Mensagem do servidor: {response_json['message']}")
                messagebox.showinfo("Cadastro", response_json['message'])
            
            # Exibir a codificação facial armazenada (se houver)
            if 'face_encoding' in response_json:
                print(f"Codificação facial armazenada: {response_json['face_encoding']}")
        else:
            print(f"Falha ao enviar a imagem. Código de status: {response.status_code}")
            print("Resposta:", response.text)
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except FileNotFoundError:
        print(f"Erro: O arquivo de imagem '{image_path}' não foi encontrado.")
        messagebox.showerror("Erro", f"Erro: Arquivo '{image_path}' não encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro: {str(e)}")
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_register_to_server(server_url):
    """
    Captura a imagem da webcam e, após o usuário confirmar, envia a imagem para o servidor.
    """
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    print("Aguardando até que um rosto seja detectado...")

    stop = False
    face_detected = False
    temp_image_path = None  # Caminho temporário da imagem capturada

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        
        if video_capture.isOpened():
            video_capture.release()
        
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        
        root.destroy()

    def on_register():
        nonlocal face_detected, temp_image_path
        user_id = entry_user_id.get()
        person_name = entry_name.get()  # Pegando o nome do usuário

        if not user_id:
            messagebox.showerror("Erro", "O User ID não pode estar vazio.")
            return

        if not person_name:
            messagebox.showerror("Erro", "O nome não pode estar vazio.")
            return
        
        if temp_image_path:
            register_face_to_server(server_url, user_id, person_name, temp_image_path)
            face_detected = False  # Resetar para permitir nova captura
            reset_gui()
        else:
            messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")

    def on_cancel():
        nonlocal face_detected, temp_image_path
        face_detected = False

        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
                temp_image_path = None
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
            except Exception as e:
                print(f"Erro ao remover o arquivo: {e}")

        reset_gui()

    def reset_gui():
        entry_user_id.delete(0, tk.END)  # Limpar o campo de User ID
        entry_name.delete(0, tk.END)  # Limpar o campo de Nome
        btn_register.pack_forget()  # Ocultar o botão de cadastrar
        btn_cancel.pack_forget()  # Ocultar o botão de cancelar
        update_frame()  # Retomar a captura da webcam

    root = tk.Tk()
    root.title("Cadastro de Rosto")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    # Campo para o User ID
    tk.Label(frame_controls, text="User ID:").pack(side=tk.LEFT)
    entry_user_id = tk.Entry(frame_controls)
    entry_user_id.pack(side=tk.LEFT)

    # Campo para o Nome da pessoa
    tk.Label(frame_controls, text="Nome:").pack(side=tk.LEFT)
    entry_name = tk.Entry(frame_controls)
    entry_name.pack(side=tk.LEFT)

    btn_cancel = tk.Button(frame_controls, text="Cancelar", command=on_cancel)
    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=on_register)

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def update_frame():
        nonlocal face_detected, temp_image_path

        if stop or face_detected:
            return

        ret, frame = video_capture.read()

        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                face_locations.append((startY, endX, endY, startX))

                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")

            _, temp_image_path = tempfile.mkstemp(suffix='.png')
            cv2.imwrite(temp_image_path, frame)

            # Fechar o arquivo corretamente
            os.close(_)

            face_detected = True
            btn_register.pack(side=tk.LEFT, padx=5)
            btn_cancel.pack(side=tk.LEFT, padx=5)
        else:
            label.after(10, update_frame)

    update_frame()
    root.mainloop()

# Testar a função com a URL de cadastro
server_url = 'http://127.0.0.1:5000/register'
detect_faces_from_webcam_and_register_to_server(server_url)


Aguardando até que um rosto seja detectado...
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Imagem enviada com sucesso: C:\Users\joaov\AppData\Local\Temp\tmp_5dt3sfh.png
Mensagem do servidor: Registered user with ID: 37428148855 and image: tmp_5dt3sfh.png
Codificação facial armazenada: [-0.17339177429676056, 0.06993291527032852, -0.011285629123449326, -0.0289751123636961, -0.08218123018741608, -0.03051603026688099, -0.012445015832781792, -0.08050430566072464, 0.10461458563804626, -0.1317499577999115, 0.15720541775226593, -0.006564435549080372, -0.2496633678674698, -0.050301261246204376, 0.013426045887172222, 0.07854659855365753, -0.016776081174612045, -0.15204861760139465, -0.1549413651227951, -0.17952653765678406, -0.030427128076553345, 0.047954026609659195, -0.025301944464445114, 0.0498966388404